# ***Facebook/bart-large-mnli zero-shot accuracy***

In [3]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score

# Paths to your filtered TSV files
SLANG_FILE = "/content/slang_OpenSub_filtered.tsv"
NON_SLANG_FILE = "/content/slang_OpenSub_negatives_filtered.tsv"

# 1) Load both datasets
df_slang = pd.read_csv(SLANG_FILE, sep="\t", dtype=str)
df_nonslang = pd.read_csv(NON_SLANG_FILE, sep="\t", dtype=str)

# 2) Assign labels: 1 = slang, 0 = non-slang
df_slang["label"] = 1
df_nonslang["label"] = 0

# 3) Concatenate into one DataFrame
df_all = pd.concat([df_slang, df_nonslang], ignore_index=True)

# 4) Extract sentences and true labels
sentences = df_all["SENTENCE"].tolist()
true_labels = df_all["label"].tolist()

# 5) Load tokenizer & model (PyTorch) for zero-shot (MNLI) classification
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

candidate_labels = ["slang", "non-slang"]

def zero_shot_classify(sentence: str, labels: list[str]) -> int:
    """
    Returns 1 if 'slang' is predicted, 0 if 'non-slang' is predicted.
    We encode each (sentence, hypothesis) pair, run through the MNLI head,
    and pick the label whose entailment logit is highest.
    """
    # Build hypotheses like "This example is slang." and "This example is non-slang."
    hypotheses = [f"This example is {lbl}." for lbl in labels]
    # Tokenize: pair each hypothesis with the same sentence
    encoded = tokenizer(
        [sentence] * len(hypotheses),
        hypotheses,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    ).to(device)

    with torch.no_grad():
        outputs = model(**encoded)
        logits = outputs.logits  # shape: (len(labels), 3)
        # For BART-MNLI, the label mapping is [contradiction, neutral, entailment]
        entail_logits = logits[:, 2]  # get "entailment" logit for each hypothesis
        probs = F.softmax(entail_logits, dim=0)  # softmax over the entailment scores
        best_idx = torch.argmax(probs).item()

    return 1 if labels[best_idx] == "slang" else 0

# 6) Run all sentences through zero-shot classifier
predicted_labels = []
for sent in sentences:
    pred = zero_shot_classify(sent, candidate_labels)
    predicted_labels.append(pred)

# 7) Compute and print accuracy
acc = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {acc:.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

cuda


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Accuracy: 0.6536


# ***BERT-base-MNLI: textattack/bert-base-uncased-MNLI, RoBERTa-large:roberta-large-mnli zero-shot accuracy***


In [3]:
# 📒 0. Colab setup – one-time
!pip install -q transformers scikit-learn tqdm  # <1 min

# 📒 1. Imports & file paths
import pandas as pd, torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

SLANG_FILE      = "/content/slang_OpenSub_filtered.tsv"
NON_SLANG_FILE  = "/content/slang_OpenSub_negatives_filtered.tsv"

# 📒 2. Load data + labels
df_slang  = pd.read_csv(SLANG_FILE, sep="\t", dtype=str)
df_neg    = pd.read_csv(NON_SLANG_FILE, sep="\t", dtype=str)
df_slang["label"] = 1             # slang
df_neg["label"]   = 0             # non-slang
df = pd.concat([df_slang, df_neg], ignore_index=True)

sentences   = df["SENTENCE"].tolist()
gold_labels = df["label"].tolist()

# 📒 3. Helper: zero-shot NLI → binary label
def build_classifier(model_name):
    tok = AutoTokenizer.from_pretrained(model_name)
    mdl = AutoModelForSequenceClassification.from_pretrained(model_name)
    mdl.to(device := torch.device("cuda" if torch.cuda.is_available() else "cpu")).eval()
    return tok, mdl, device

def predict_sentence(tok, mdl, device, sent):
    hyps = ["This example is slang.", "This example is non-slang."]
    enc  = tok([sent]*2, hyps, return_tensors="pt",
               padding=True, truncation=True, max_length=256).to(device)
    with torch.no_grad():
        logits = mdl(**enc).logits[:, 2]          # entailment logit
    return int(torch.argmax(F.softmax(logits, dim=0)).item() == 0)  # 1 = slang

# 📒 4. Evaluate both models
models = {
    "BERT-base-MNLI":  "textattack/bert-base-uncased-MNLI",
    "RoBERTa-large":   "roberta-large-mnli",
}

for nick, hub_id in models.items():
    tok, mdl, dev = build_classifier(hub_id)
    preds = [predict_sentence(tok, mdl, dev, s) for s in tqdm(sentences, desc=nick)]
    acc   = accuracy_score(gold_labels, preds)
    print(f"{nick:20s}  accuracy = {acc:.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RoBERTa-large:   0%|          | 0/3617 [00:00<?, ?it/s]

RoBERTa-large         accuracy = 0.4487


# ***DeBERTa-v3-large (MNLI-fine-tuned) zero-shot accuracy***


In [4]:
# 📝 Imports & paths
import pandas as pd, torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

SLANG_FILE      = "/content/slang_OpenSub_filtered.tsv"
NON_SLANG_FILE  = "/content/slang_OpenSub_negatives_filtered.tsv"

# 📂 Load data
df_slang = pd.read_csv(SLANG_FILE, sep="\t", dtype=str)
df_neg   = pd.read_csv(NON_SLANG_FILE, sep="\t", dtype=str)
df_slang["label"] = 1      # slang
df_neg["label"]   = 0      # non-slang
df = pd.concat([df_slang, df_neg], ignore_index=True)

sentences   = df["SENTENCE"].tolist()
gold_labels = df["label"].tolist()

# 🔍 Load DeBERTa-v3-large (MNLI-fine-tuned)
MODEL_ID = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
tok  = AutoTokenizer.from_pretrained(MODEL_ID)
mdl  = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mdl.to(device).eval()

# 🧮 Zero-shot NLI → slang / non-slang
def predict(sent: str) -> int:
    hyps = ["This example is slang.", "This example is non-slang."]
    enc  = tok([sent]*2, hyps, return_tensors="pt",
               padding=True, truncation=True, max_length=256).to(device)
    with torch.no_grad():
        logits = mdl(**enc).logits[:, 2]              # entailment logit
    return int(torch.argmax(F.softmax(logits, dim=0)).item() == 0)  # 1 = slang

# ⚡ Inference loop
preds = [predict(s) for s in tqdm(sentences, desc="DeBERTa-v3-large")]

# 📊 Accuracy
print(f"Accuracy: {accuracy_score(gold_labels, preds):.4f}")


tokenizer_config.json:   0%|          | 0.00/395 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/870M [00:00<?, ?B/s]

DeBERTa-v3-large:   0%|          | 0/3617 [00:00<?, ?it/s]

Accuracy: 0.5939
